# Video styling

In [1]:
from tqdm import tqdm
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
import cv2 as cv2 # opencv computer vision library
from PIL import Image
import cv2
import os
import sys
from typing import Dict, Any

dirname = os.path.abspath(os.path.join(os.getcwd(), "..", "lib"))
sys.path.append(dirname)

from display import imshow
from vgg import VGGFeatures
from utils import get_file_type
from data import ImageFolderDataset
from DeepStyleX import DeepStyleX

import moviepy as mp

## Import video and image

In [2]:
video_path = '../data/example_video.mp4'
image_path = "../data/Central-Park.jpg"


print(video_path,get_file_type(video_path))
print(image_path,get_file_type(image_path))

../data/example_video.mp4 video
../data/Central-Park.jpg image


For handling videos

In [3]:
def clip_frame_iterator(video_clip, batch_size):
    """
    Generator to yield batches of frames from a video.

    Args:
        video_path (str): Path to the input video.
        batch_size (int): Number of frames per batch.

    Yields:
        list: A batch of frames (as numpy arrays).
    """
    batch = []

    for frame in video_clip.iter_frames(fps=video_clip.fps, dtype="uint8"):
        batch.append(frame)
        if len(batch) == batch_size:
            yield torch.from_numpy(np.stack(batch, axis=0)).permute(0, 3, 1, 2).float()  # Yield a full batch
            batch = []

    if len(batch)>0:
        yield torch.from_numpy(np.stack(batch, axis=0)).permute(0, 3, 1, 2).float()  # Yield a partial batch

In [4]:
model, _ = DeepStyleX.load('../data/saves/abstract.dsx')
model.eval()
print("Model set to evaluation")

Model set to evaluation


In [5]:
batch_size = 3

In [6]:
def new_video_generator(clip, batch_size):
    total_frames = int(clip.duration * clip.fps+0.1)
    for batch in tqdm(clip_frame_iterator(clip, batch_size=batch_size), total=(total_frames+batch_size-1)//batch_size):
        with torch.no_grad():
            output = model(batch)
            output = output.permute(0, 2, 3, 1).cpu().detach().numpy().astype(np.uint8)
        for frame in output:
            yield frame

In [7]:
original_video = mp.VideoFileClip(video_path)
original_audio = original_video.audio

In [8]:
# Create the video clip
clip = mp.ImageSequenceClip(list(new_video_generator(original_video, batch_size=4)), fps=original_video.fps)
clip.duration

100%|██████████| 81/81 [02:43<00:00,  2.01s/it]


10.733333333333333

In [9]:
clip = clip.with_audio(original_audio)

In [10]:
output_path = "../output/video.mp4"

In [11]:
clip.write_videofile(output_path, codec="libx264", audio_codec="aac")

MoviePy - Building video ../output/video.mp4.
MoviePy - Writing audio in videoTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
MoviePy - Writing video ../output/video.mp4



MoviePy - Done !
MoviePy - video ready ../output/video.mp4
